In [2]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
market = Market()
strategy = Strategy()

In [4]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()
strategy.connect()
quarterly_sim = strategy.retrieve("quarterly_sim")
weekly_sim = strategy.retrieve("weekly_sim")
strategy.disconnect()

In [5]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [22]:
trades = []
analysis = []
number_of_training_weeks = 14
weekly_model_training_year = 5
weekly_category_training_year = 1
quarterly_model_training_year = 3
quarterly_category_training_year = 1
# for number_of_training_weeks in tqdm(range(14,69,14)):
#     for weekly_model_training_year in tqdm(range(1,6)):
#         for weekly_category_training_year in tqdm(range(1,5)):
current_weekly_sim = weekly_sim[(weekly_sim["mty"]==weekly_model_training_year) & \
                               (weekly_sim["tw"]==number_of_training_weeks) & \
                               (weekly_sim["cty"]==weekly_category_training_year)]
#             for quarterly_category_training_year in tqdm(range(1,5)):
#                 for quarterly_model_training_year in tqdm(range(3,6)):
current_sim = quarterly_sim[(quarterly_sim["model_training_year"]==quarterly_model_training_year) & (quarterly_sim["category_training_year"]==quarterly_category_training_year)]
final_sim = prices.merge(current_sim,on=["year","quarter","ticker"],how="left").dropna().reset_index()
final_sim = final_sim[["year","quarter","week","date","ticker","adjClose","quarterly_price_regression_prediction","score"]]
final_sim.rename(columns={"score":"quarterly_score"},inplace=True)
final_sim["quarterly_delta"] = (final_sim["quarterly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
final_sim = final_sim.merge(current_weekly_sim,on=["year","quarter","week","ticker"],how="left")
final_sim["weekly_delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
final_sim.rename(columns={"score":"weekly_score"},inplace=True)
start = final_sim["date"].min()
end = final_sim["date"].max()
date = start
seats = 11
final_sim.sort_values("date",ascending=True,inplace=True)
analysis.append({"ntw":number_of_training_weeks,
                "wmty":weekly_model_training_year,
                "wcty":weekly_category_training_year,
      "qmty":quarterly_model_training_year,
      "qcty":quarterly_category_training_year,
     "index_size":final_sim.index.size})
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in range(60,100,10):
        date = start
        while date <= end:
            req = float(score/100)
            if date.weekday() > 1:
                date = date + timedelta(days=1)
            try:
                todays_sim = final_sim[(final_sim["date"]==date) & (final_sim["weekly_delta"] > 0) \
                                      & (final_sim["weekly_score"] >= req)].sort_values("weekly_delta",ascending=False)
                if todays_sim.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for seat in range(min(seats,todays_sim.index.size)):
                        try:
                            trade = todays_sim.iloc[seat]
                            ticker = trade["ticker"]
                            buy_price = trade["adjClose"]
                            exit_price = buy_price * (1+delta)
                            max_date = date + timedelta(days=30)
                            exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                (final_sim["date"] > date) & \
                                              (final_sim["date"] <= max_date) & \
                                              (final_sim["adjClose"] >= exit_price)
                                            ]
                            if exits.index.size < 1:
                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                              (final_sim["year"] == date.year) & \
                                              (final_sim["adjClose"] >= buy_price)].sort_values("date",ascending=False)
                                if exits.index.size > 1:
                                    sell_trade = exits.iloc[0]
                                else:
                                    exits = final_sim[(final_sim["ticker"]==ticker) & 
                                              (final_sim["year"] == date.year)].sort_values("date",ascending=False)
                                    sell_trade = exits.iloc[exits.index.size -1]
                            else:
                                sell_trade = exits.iloc[0]
                            trade["sell_date"] = sell_trade["date"]
                            trade["sell_price"] = sell_trade["adjClose"]
                            trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                            trade["score"] = score
                            trade["d"] = d
                            trade["qcty"] = quarterly_category_training_year
                            trade["qmty"] = quarterly_model_training_year
                            trade["wcty"] = weekly_category_training_year
                            trade["wmty"] = weekly_model_training_year
                            trade["tw"] = number_of_training_weeks
                            trade["seat"] = seat
                            trades.append(trade)
                            date = sell_trade["date"] + timedelta(days=1)
                        except Exception as e:
                            continue
                    date = date + timedelta(days=1)
            except Exception as e:
                date = date + timedelta(days=1)

  0%|                                                                                                                                             | 0/3 [00:00<?, ?it/s]C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)
100%|█████████████████████████████████████████████████████████████

In [24]:
t = pd.DataFrame(trades)

In [25]:
a = []
seats = 10
number_of_training_weeks = 14
weekly_model_training_year = 5
weekly_category_training_year = 1
quarterly_model_training_year = 3
quarterly_category_training_year = 1
# for category_training_year in tqdm(range(1,5)):
#     for model_training_year in tqdm(range(3,6)):
for delta in tqdm(range(0,15,5)):
    d = delta/100
    for score in tqdm(range(70,100,10)):
        score_trades = t[(t["score"]==score) & 
                         (t["d"]==d) & 
                         (t["qcty"]==quarterly_category_training_year) & 
                         (t["qmty"]==quarterly_model_training_year) &
                        (t["wcty"]==weekly_category_training_year) & 
                         (t["wmty"]==weekly_model_training_year) &
                        (t["tw"]==number_of_training_weeks)]
        cash = []
        for seat in range(seats):
            initial = float(100 / seats)
            seat_trades = score_trades[score_trades["seat"]==seat]
            for delta in seat_trades["delta"]:
                initial = initial * (1+delta)
                cash.append(initial)
            analysis = {}
            analysis["score"] = score
            analysis["trades"] = seat_trades.index.size
            if seat_trades.index.size > 0:
                analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
                seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
                seat_trades["hpr"] =[x.days for x in seat_trades["hpr"]]
                analysis["hpr"] = seat_trades["hpr"].mean()
                analysis["seat"] = seat
                analysis["d"] = d
                analysis["qcty"] = quarterly_category_training_year
                analysis["qmty"] = quarterly_model_training_year
                analysis["wcty"] = weekly_category_training_year
                analysis["wmty"] = weekly_model_training_year
                analysis["tw"] = number_of_training_weeks
                analysis["pv"] = initial
                a.append(analysis)

  0%|                                                                                                                                             | 0/3 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_5436/3492107585.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
C:\Users\ericj\AppData\Local\Temp/ipykernel_5436/3492107585.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
C:\Users\ericj\AppData\Local\Temp/ipykernel_5436/3492107585.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [26]:
final = pd.DataFrame(a)

In [27]:
final

,score,trades,winrate,hpr,seat,d,qcty,qmty,wcty,wmty,tw,pv
0,70,17,0.941176,7.235294,0,0.0,1,3,1,5,14,19.297805
1,70,17,1.000000,9.705882,1,0.0,1,3,1,5,14,24.989287
2,70,17,0.941176,19.411765,2,0.0,1,3,1,5,14,22.950099
3,70,17,0.941176,21.529412,3,0.0,1,3,1,5,14,20.639068
4,70,17,0.941176,25.470588,4,0.0,1,3,1,5,14,20.961982
...,...,...,...,...,...,...,...,...,...,...,...,...
85,90,7,0.714286,145.714286,5,0.1,1,3,1,5,14,24.193034
86,90,7,0.428571,154.571429,6,0.1,1,3,1,5,14,15.084874
87,90,7,0.714286,135.000000,7,0.1,1,3,1,5,14,14.587421
88,90,7,1.000000,152.714286,8,0.1,1,3,1,5,14,15.494949


In [29]:
hl = final.pivot_table(index=["score","d","qcty","qmty","wcty","wmty","tw"],columns="seat",values="pv")

In [30]:
report = final.merge(hl.reset_index(),on=["score","d","qcty","qmty","wcty","wmty","tw"]).drop(["pv","seat"],axis=1).groupby(["score","d","qcty","qmty","wcty","wmty","tw"]).mean()
report["pv"] = [sum([row[1][x] for x in range(seats)]) for row in report.iterrows()]

In [31]:
report.sort_values("pv",ascending=False).reset_index()

,score,d,qcty,qmty,wcty,wmty,tw,trades,winrate,hpr,...,1,2,3,4,5,6,7,8,9,pv
0,70,0.00,1,3,1,5,14,16.5,0.957721,29.274167,...,24.989287,22.950099,20.639068,20.961982,23.151162,30.793764,43.659495,41.202380,83.837195,331.482238
1,80,0.00,1,3,1,5,14,16.5,0.957721,29.274167,...,24.989287,22.950099,20.639068,20.961982,23.151162,30.793764,43.659495,41.202380,83.837195,331.482238
2,90,0.00,1,3,1,5,14,16.5,0.957721,29.274167,...,24.989287,22.950099,20.639068,20.961982,23.151162,30.793764,43.659495,41.202380,83.837195,331.482238
3,70,0.05,1,3,1,5,14,7.0,0.742857,122.957143,...,16.852335,20.095291,16.274180,19.759895,24.193034,15.084874,14.587421,15.494949,51.131132,210.686196
4,70,0.10,1,3,1,5,14,7.0,0.742857,122.957143,...,16.852335,20.095291,16.274180,19.759895,24.193034,15.084874,14.587421,15.494949,51.131132,210.686196
5,80,0.05,1,3,1,5,14,7.0,0.742857,122.957143,...,16.852335,20.095291,16.274180,19.759895,24.193034,15.084874,14.587421,15.494949,51.131132,210.686196
6,80,0.10,1,3,1,5,14,7.0,0.742857,122.957143,...,16.852335,20.095291,16.274180,19.759895,24.193034,15.084874,14.587421,15.494949,51.131132,210.686196
7,90,0.05,1,3,1,5,14,7.0,0.742857,122.957143,...,16.852335,20.095291,16.274180,19.759895,24.193034,15.084874,14.587421,15.494949,51.131132,210.686196
8,90,0.10,1,3,1,5,14,7.0,0.742857,122.957143,...,16.852335,20.095291,16.274180,19.759895,24.193034,15.084874,14.587421,15.494949,51.131132,210.686196


In [34]:
score = report.sort_values("pv",ascending=False).reset_index().iloc[0]["score"].item()
d = report.sort_values("pv",ascending=False).reset_index().iloc[0]["d"].item()
qcty = report.sort_values("pv",ascending=False).reset_index().iloc[0]["qcty"].item()
qmty = report.sort_values("pv",ascending=False).reset_index().iloc[0]["qmty"].item()
wcty = report.sort_values("pv",ascending=False).reset_index().iloc[0]["wcty"].item()
wmty = report.sort_values("pv",ascending=False).reset_index().iloc[0]["wmty"].item()
tw = report.sort_values("pv",ascending=False).reset_index().iloc[0]["tw"].item()
benchmark = t[(t["score"]==score) & (t["d"]==d) & (t["qmty"]==qmty) & (t["qcty"]==qcty)  
              & (t["wmty"]==wmty) & (t["wcty"]==wcty)
              & (t["tw"]==tw)]

In [35]:
timelines =[]
for seat in tqdm(range(seats)):
    seat_trades = benchmark[benchmark["seat"]==seat]
    initial = float(100/seats)
    timeline = []
    for row in seat_trades.iterrows():
        initial = initial * (1+row[1]["delta"])
        timeline.append(initial)
    seat_trades["pv"] = timeline
    timelines.append(seat_trades)

  0%|                                                                                                                                            | 0/10 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_5436/744403605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["pv"] = timeline
C:\Users\ericj\AppData\Local\Temp/ipykernel_5436/744403605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seat_trades["pv"] = timeline
C:\Users\ericj\AppData\Local\Temp/ipykernel_5436/744403605.py:9:

In [36]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()
sp5.rename(columns={"Symbol":"ticker"},inplace=True)
benchmark = benchmark.merge(sp5,on="ticker",how="left")

In [37]:
sp5.columns

Index(['_id', 'ticker', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [38]:
# industry check
benchmark.groupby(["year","quarter","GICS Sector"]).mean().sort_values("delta",ascending=False).reset_index()

,year,quarter,GICS Sector,week,adjClose,quarterly_price_regression_prediction,quarterly_score,quarterly_delta,weekly_price_regression_prediction,weekly_score,...,sell_price,delta,score,d,qcty,qmty,wcty,wmty,seat,CIK
0,2020,2,Consumer Discretionary,17.857143,12.317434,42.128177,0.969356,2.704057,13.789585,0.966964,...,27.786267,1.026496,70.0,0.0,1.0,3.0,1.0,5.0,5.142857,9.261876e+05
1,2020,2,Materials,17.000000,45.968661,68.280716,0.969356,0.485375,50.757953,0.979505,...,60.838410,0.323476,70.0,0.0,1.0,3.0,1.0,5.0,6.000000,1.489393e+06
2,2021,1,Materials,1.000000,103.349056,119.653816,0.726359,0.157764,110.404465,0.998987,...,133.336805,0.290160,70.0,0.0,1.0,3.0,1.0,5.0,9.000000,5.125300e+04
3,2019,1,Consumer Discretionary,6.000000,296.070548,230.321058,0.959809,-0.236012,303.466347,0.999052,...,335.124802,0.212447,70.0,0.0,1.0,3.0,1.0,5.0,4.500000,9.436570e+05
4,2019,1,Information Technology,1.000000,37.579196,50.249007,0.959809,0.139805,40.818881,0.997411,...,40.731615,0.127188,70.0,0.0,1.0,3.0,1.0,5.0,0.500000,3.307500e+03
5,2020,2,Health Care,15.000000,50.476229,120.865726,0.969356,1.349746,51.460950,0.988118,...,55.915189,0.107675,70.0,0.0,1.0,3.0,1.0,5.0,4.000000,5.325155e+05
6,2021,1,Consumer Staples,7.000000,44.183804,57.995438,0.726359,0.312595,46.877384,0.998987,...,48.850408,0.105618,70.0,0.0,1.0,3.0,1.0,5.0,7.000000,2.454500e+04
7,2019,1,Real Estate,1.000000,218.669457,218.876801,0.959809,0.000948,226.686983,0.999222,...,240.514337,0.099899,70.0,0.0,1.0,3.0,1.0,5.0,8.000000,9.205220e+05
8,2020,2,Industrials,18.428571,34.257941,83.668328,0.969356,2.707953,37.411676,0.973896,...,35.975112,0.090571,70.0,0.0,1.0,3.0,1.0,5.0,2.428571,4.842586e+04
9,2021,3,Energy,31.727273,88.114958,64.945414,0.952783,-0.203081,93.596754,0.997674,...,94.979133,0.082234,70.0,0.0,1.0,3.0,1.0,5.0,5.909091,1.130636e+06


In [39]:
benchmark.groupby(["year","quarter","ticker"]).mean().sort_values("delta",ascending=False).reset_index().merge(sp5[["ticker","Security"]],on="ticker",how="left").head(20)

,year,quarter,ticker,week,adjClose,quarterly_price_regression_prediction,quarterly_score,quarterly_delta,weekly_price_regression_prediction,weekly_score,...,delta,score,d,qcty,qmty,wcty,wmty,seat,CIK,Security
0,2020,2,CZR,17.0,15.580000,47.456230,0.969356,2.045971,17.169388,0.979505,...,1.684852,70.0,0.0,1.0,3.0,1.0,5.0,8.500000,1590895.0,Caesars Entertainment
1,2020,2,LYB,17.0,45.968661,68.280716,0.969356,0.485375,50.757953,0.979505,...,0.323476,70.0,0.0,1.0,3.0,1.0,5.0,6.000000,1489393.0,LyondellBasell
2,2019,1,SBUX,6.0,64.221097,47.921078,0.959809,-0.253811,64.812803,0.999222,...,0.315294,70.0,0.0,1.0,3.0,1.0,5.0,9.000000,829224.0,Starbucks
3,2021,1,IFF,1.0,103.349056,119.653816,0.726359,0.157764,110.404465,0.998987,...,0.290160,70.0,0.0,1.0,3.0,1.0,5.0,9.000000,51253.0,International Flavors & Fragrances
4,2019,1,CTLT,1.0,29.840000,70.169312,0.959809,1.351518,30.756996,0.998103,...,0.222855,70.0,0.0,1.0,3.0,1.0,5.0,9.000000,1596783.0,Catalent
5,2019,1,AMD,1.0,17.050000,13.274443,0.959809,-0.221440,17.888403,0.995601,...,0.206452,70.0,0.0,1.0,3.0,1.0,5.0,1.000000,2488.0,Advanced Micro Devices
6,2021,2,CTLT,20.5,101.245000,94.994041,0.980764,-0.061687,107.733647,0.998609,...,0.196678,70.0,0.0,1.0,3.0,1.0,5.0,4.500000,1596783.0,Catalent
7,2019,1,NSC,1.0,136.964893,157.153992,0.959809,0.147403,140.993514,0.999222,...,0.169143,70.0,0.0,1.0,3.0,1.0,5.0,10.000000,702165.0,Norfolk Southern
8,2020,2,RE,17.0,170.277154,235.287262,0.969356,0.381790,188.373629,0.986892,...,0.163640,70.0,0.0,1.0,3.0,1.0,5.0,5.000000,1095073.0,Everest Re
9,2021,3,EOG,33.0,64.260000,55.441879,0.934215,-0.137226,69.320049,0.998756,...,0.155462,70.0,0.0,1.0,3.0,1.0,5.0,10.000000,821189.0,EOG Resources
